[Reference](https://medium.com/top-python-libraries/mastering-python-logging-for-production-grade-applications-574ec55f99f9)

# 1. The Basics: Moving Beyond print()

In [1]:
import logging

# Basic config
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.debug("Debugging details here")
logging.info("Service started successfully")
logging.warning("Low memory detected")
logging.error("Failed to connect to database")
logging.critical("System crash imminent")

ERROR:root:Failed to connect to database
CRITICAL:root:System crash imminent


# 2. Writing Logs to Files (Rotating for Safety)

In [2]:
import logging
from logging.handlers import RotatingFileHandler

logger = logging.getLogger("my_app")
logger.setLevel(logging.DEBUG)

# Rotate logs after 5MB, keep 3 backups
handler = RotatingFileHandler("app.log", maxBytes=5_000_000, backupCount=3)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)

logger.addHandler(handler)

logger.info("Application log rotation configured successfully")

INFO:my_app:Application log rotation configured successfully


# 3. Structured Logging with JSON

In [3]:
import logging
import json

class JsonFormatter(logging.Formatter):
    def format(self, record):
        log_record = {
            "time": self.formatTime(record, "%Y-%m-%d %H:%M:%S"),
            "level": record.levelname,
            "message": record.getMessage(),
            "logger": record.name
        }
        return json.dumps(log_record)

logger = logging.getLogger("structured_app")
handler = logging.StreamHandler()
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)

logger.warning("Disk space running low")

{"time": "2025-09-06 14:05:17", "level": "WARNING", "message": "Disk space running low", "logger": "structured_app"}


# 4. Adding Context with Filters

In [4]:
class RequestIDFilter(logging.Filter):
    def __init__(self, request_id):
        super().__init__()
        self.request_id = request_id

    def filter(self, record):
        record.request_id = self.request_id
        return True

logger = logging.getLogger("context_logger")
handler = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s - %(levelname)s - [RequestID: %(request_id)s] - %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

# Add filter dynamically
logger.addFilter(RequestIDFilter("abc-123"))

logger.info("User fetched profile data")

# 5. Asynchronous Logging for Performance

In [5]:
import logging
import logging.handlers
import queue
import threading

log_queue = queue.Queue(-1)

queue_handler = logging.handlers.QueueHandler(log_queue)
queue_listener = logging.handlers.QueueListener(log_queue, logging.StreamHandler())

logger = logging.getLogger("async_logger")
logger.addHandler(queue_handler)
logger.setLevel(logging.DEBUG)

queue_listener.start()
logger.info("Non-blocking logging in action")

INFO:async_logger:Non-blocking logging in action
Non-blocking logging in action


# 6. Remote Logging (Centralized Monitoring)

In [7]:
import logging
import logging.handlers

logger = logging.getLogger("remote_logger")
logger.setLevel(logging.INFO)

syslog_handler = logging.handlers.SysLogHandler(address=("logs.mycompany.com", 514))
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
syslog_handler.setFormatter(formatter)

logger.addHandler(syslog_handler)

logger.info("Remote logging configured")

# 7. Color-Coding Logs for Humans

In [9]:
import logging
import coloredlogs

logger = logging.getLogger("colorful_logger")
coloredlogs.install(level="DEBUG", logger=logger)

logger.debug("This is debug")
logger.info("This is info")
logger.warning("This is warning")
logger.error("This is error")
logger.critical("This is critical")

# 8. Real-World Example: Logging in a FastAPI App

In [10]:
from fastapi import FastAPI, Request
import logging

app = FastAPI()
logger = logging.getLogger("api")
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

@app.middleware("http")
async def log_requests(request: Request, call_next):
    logger.info(f"Incoming request: {request.method} {request.url}")
    response = await call_next(request)
    logger.info(f"Response status: {response.status_code}")
    return response

@app.get("/")
async def root():
    logger.debug("Processing root request")
    return {"message": "Hello World"}